In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
import tensorflow as tf
import malaya_speech
from malaya_speech.train.model import univnet
from malaya_speech.train.model import universal_melgan as melgan
import numpy as np

In [4]:
gen_config = univnet.GeneratorConfig()
gen_config.channel_size = 16
melgan_config = malaya_speech.config.universal_melgan_config

In [5]:
generator = univnet.Generator(gen_config)
# discriminator = melgan.MultiScaleDiscriminator(
#     melgan.WaveFormDiscriminatorConfig(
#         **melgan_config['melgan_waveform_discriminator_params']
#     ),
#     melgan.STFTDiscriminatorConfig(
#         **melgan_config['melgan_stft_discriminator_params']
#     ),
#     name='universalmelgan-discriminator',
# )

In [6]:
y = tf.placeholder(tf.float32, (None, None))
x = tf.placeholder(tf.float32, (None, None, 80))

In [7]:
y_hat = generator(x)
# p_hat = discriminator(y_hat)
# p = discriminator(tf.expand_dims(y, -1))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [8]:
from malaya_speech.train.loss import calculate_2d_loss, calculate_3d_loss

In [9]:
mse_loss = tf.keras.losses.MeanSquaredError()
mae_loss = tf.keras.losses.MeanAbsoluteError()

In [10]:
# adv_loss = 0.0
# for i in range(len(p_hat)):
#     adv_loss += mse_loss(tf.ones_like(p_hat[i][-1]), p_hat[i][-1])
    
# adv_loss /= i + 1
# fm_loss = 0.0
# for i in range(len(p_hat)):
#     for j in range(len(p_hat[i]) - 1):
#         fm_loss += mae_loss(p[i][j], p_hat[i][j])
        
# fm_loss /= (i + 1) * (j + 1)
# adv_loss += 10 * fm_loss

In [11]:
# real_loss = 0.0
# fake_loss = 0.0
# for i in range(len(p)):
#     real_loss += mse_loss(tf.ones_like(p[i][-1]), p[i][-1])
#     fake_loss += mse_loss(tf.zeros_like(p_hat[i][-1]), p_hat[i][-1])
    
# real_loss /= i + 1
# fake_loss /= i + 1
# dis_loss = real_loss + fake_loss

In [12]:
# adv_loss, dis_loss

In [13]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [14]:
%%time

sess.run(y_hat, feed_dict = {x: np.random.uniform(size=(1,200,80))}).shape

CPU times: user 1.55 s, sys: 186 ms, total: 1.74 s
Wall time: 912 ms


(1, 51200, 1)

In [15]:
%%time

sess.run(y_hat, feed_dict = {x: np.random.uniform(size=(1,210,80))}).shape

CPU times: user 728 ms, sys: 175 ms, total: 903 ms
Wall time: 167 ms


(1, 53760, 1)

In [16]:
from malaya_speech.train.model.utils import shape_list

In [17]:
total_trainable_params = 0
for v in tf.trainable_variables():
    total_trainable_params += np.prod(shape_list(v))
total_trainable_params

7929331

In [18]:
# generator.remove()

In [19]:
# total_trainable_params = 0
# for v in tf.trainable_variables():
#     total_trainable_params += np.prod(shape_list(v))
# total_trainable_params

In [20]:
saver = tf.train.Saver()
saver.save(sess, 'test/model.ckpt')

'test/model.ckpt'

In [21]:
!ls -lh test

total 70008
-rw-r--r--  1 huseinzolkepli  staff    77B May  3 21:39 checkpoint
-rw-r--r--  1 huseinzolkepli  staff    30M May  3 21:39 model.ckpt.data-00000-of-00001
-rw-r--r--  1 huseinzolkepli  staff    11K May  3 21:39 model.ckpt.index
-rw-r--r--  1 huseinzolkepli  staff   3.9M May  3 21:39 model.ckpt.meta


In [22]:
!rm -rf test